# !!!DEPRECATED!!!!

In [39]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import re

# Scraping missing actor data
In this notebook, we are scrappping missing actor data for our data story: analysing the ideal actor profile for each genre over time.

The missing data is:
1. height of every actor
2. nationality of every actor
3. ...

## Import actor data from imdb


In [3]:
# get imdb actor list (source: https://datasets.imdbws.com/)
actors = pd.read_csv('./data/name.basics.tsv/data.tsv', sep='\t')
actors.head()

NameError: name 'pd' is not defined

In [ ]:
actors.shape

## Scraping the height and birthplace (=nationality) of every actor
Since we regard the birthplace as the decider for the nationality of an actor, we definitely determine some actors' nationality wrong. However, due to the work bound to finding the proper nationality exceeds this project's frame and the fact that the birthplace in mostcases reflects the nationality, we decided to neglect this error.

In [52]:
# input string of imdb actor id and output is an array of birthday, birthplace, height (in this order)
def getImdbActorInfo(imdbActorId):
    try:
        page = requests.get('https://www.imdb.com/name/'+imdbActorId+'/bio/')
        soup = BeautifulSoup(page.text, 'html.parser')
    
        table = soup.find('table', attrs={'id':'overviewTable'})
        birthday = table.find("time")['datetime']
        birthplace = "".join([a.contents for a in table.find_all("a", href=True) if "birth_place" in a["href"]][0])
        height = int(re.findall(r'\(.*?\)', table.find_all('td')[[idx for idx, td in enumerate(table.find_all('td')) if "Height" in td.contents][0]+1].contents[0])[0].replace("(", "").replace(")", "").replace(".", "").replace("\xa0", "").replace("m", ""))
        return [birthday, birthplace, height]
    except:
        return "NA"

In [53]:
#test upper function
print(getImdbActorInfo("nm0000226"))

['1968-9-25', 'Philadelphia, Pennsylvania, USA', 187]


In [58]:
# scrape for personal info of actor on imdb
# TODO make more efficient takes way too long time
birthdays = []
birthplaces = []
heights = []
for idx, actor in enumerate(actors.loc[100, "nconst"]):
    info = getImdbActorInfo(actor)
    if info == "NA": 
        birthdays.append("NA")
        birthplaces.append("NA")
        heights.append("NA")
    else:
        birthdays.append(info[0])
        birthplaces.append(info[1])
        heights.append(info[2])
actors["birthday"] = birthdays
actors["birthplace"] = birthplaces
actors["height"] = heights